# Deep Research: BTC-MACD-ADX Optimization

**Navigation** : [Index](../../README.md) | [Retour](../README.md)

## Objectif

Maximiser le ratio de Sharpe sur la periode etendue (2019-2025) pour la strategie MACD+ADX sur Bitcoin.

## Vue d'ensemble de la strategie

- **Entree** : Croisement MACD confirme le trend + ADX confirme la force
- **Sortie** : Croisement MACD oppose OU ADX descend sous le seuil
- **Parametres actuels** (optimises lors de recherches precedentes) :
  - Fenetre ADX : 80 (etait 140)
  - Percentile inferieur ADX : 5 (etait 6)
  - Percentile superieur ADX : 85 (etait 86)

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Charger et analyser des donnees historiques BTC avec yfinance
2. Calculer les indicateurs MACD et ADX
3. Detecter les regimes de marche (bull/bear/sideways)
4. Implementer un moteur de backtest complet
5. Executer un grid search pour optimiser les parametres
6. Analyser la performance par regime de marche

### Prerequis

- Python 3.10+ avec pandas, numpy, matplotlib, yfinance
- Connaissance des indicateurs techniques (MACD, ADX)
- Comprehension du backtesting et des metriques de risque

### Duree estimee : 60 minutes

## Questions de recherche

1. Quelle est la fenetre ADX optimale pour differents regimes de marche ?
2. Les seuils de percentiles doivent-ils etre dynamiques (ajustables en volatilite) ?
3. Peut-on ameliorer le timing de sortie avec des stops suiveurs ?
4. Comment la strategie performe-t-elle dans differents regimes BTC ?

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Download BTC data from 2017
print("Downloading BTC data...")
btc = yf.download("BTC-USD", start="2017-01-01", end="2025-02-18", progress=False)
print(f"Data shape: {btc.shape}")
print(f"Date range: {btc.index[0]} to {btc.index[-1]}")
print(f"Total days: {len(btc)}")

# Use Close prices
close = btc['Close'].copy()
if isinstance(close, pd.DataFrame):
    close = close.iloc[:, 0]

print(f"\nBTC Price range: ${close.min():.0f} - ${close.max():.0f}")
print(f"Current price: ${close.iloc[-1]:.0f}")

### Interpretation : Donnees historiques BTC

Le chargement des donnees historiques est la base de toute analyse quantitative.

| Statistique | Valeur | Interpretation |
|-------------|--------|----------------|
| Periode | 2017-2025 | Couverture des cycles bull/bear complets |
| Prix min/max | A calculer | Volatilite extreme typique du BTC |
| Prix actuel | A calculer | Reference pour le positionnement |

**Points cles** :
- La periode 2017-2025 couvre plusieurs cycles majeurs de BTC
- L'analyse sur 8+ ans permet de valider la robustesse de la strategie
- Le warm-up pour les indicateurs (MACD, ADX) necessite au moins 100-200 jours
- Les donnees yfinance sont gratuites mais peuvent differer legèrement des donnees de trading

> **Note technique** : Pour la production sur QuantConnect, utilisez les donnees Binance BTCUSDT. La periode de recherche etendue (2019+) correspond a la disponibilite des donnees crypto sur QC.

In [ ]:
# Calculate technical indicators
def calculate_indicators(df, ema_fast=12, ema_slow=26, signal_period=9):
    """Calculate MACD and ADX indicators"""
    close = df['Close'].copy()
    if isinstance(close, pd.DataFrame):
        close = close.iloc[:, 0]
    
    high = df['High'].copy()
    if isinstance(high, pd.DataFrame):
        high = high.iloc[:, 0]
    
    low = df['Low'].copy()
    if isinstance(low, pd.DataFrame):
        low = low.iloc[:, 0]
    
    # MACD
    ema12 = close.ewm(span=ema_fast, adjust=False).mean()
    ema26 = close.ewm(span=ema_slow, adjust=False).mean()
    macd = ema12 - ema26
    signal = macd.ewm(span=signal_period, adjust=False).mean()
    histogram = macd - signal
    
    # ADX (simplified calculation)
    def calculate_adx(high, low, close, period=14):
        # True Range
        tr1 = high - low
        tr2 = abs(high - close.shift(1))
        tr3 = abs(low - close.shift(1))
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        
        # Directional Movement
        up_move = high - high.shift(1)
        down_move = low.shift(1) - low
        
        plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0)
        minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0)
        
        # Smooth TR and DM
 atr = tr.rolling(window=period).mean()
        plus_di = 100 * pd.Series(plus_dm, index=close.index).rolling(window=period).mean() / atr
        minus_di = 100 * pd.Series(minus_dm, index=close.index).rolling(window=period).mean() / atr
        
        # DX and ADX
        dx = 100 * abs(plus_di - minus_di) / (plus_di + minus_di)
        adx = dx.rolling(window=period).mean()
        
        return adx, plus_di, minus_di
    
    adx, plus_di, minus_di = calculate_adx(high, low, close)
    
    return pd.DataFrame({
        'close': close,
        'macd': macd,
        'signal': signal,
        'histogram': histogram,
        'adx': adx,
        'plus_di': plus_di,
        'minus_di': minus_di
    }, index=close.index)

# Calculate indicators
data = calculate_indicators(btc)
print("Indicators calculated successfully")
print(f"ADX mean: {data['adx'].mean():.2f}")
print(f"ADX std: {data['adx'].std():.2f}")
print(f"ADX 50th percentile: {data['adx'].quantile(0.5):.2f}")
print(f"ADX 85th percentile: {data['adx'].quantile(0.85):.2f}")

### Interpretation : Indicateurs MACD et ADX

Le calcul des indicateurs techniques est la premiere etape de l'analyse quantitative.

| Indicateur | Description | Utilite dans la strategie |
|------------|-------------|---------------------------|
| MACD | Moving Average Convergence Divergence | Detect les changements de tendance |
| Signal | EMA du MACD | Filtre pour confirmer les croisements |
| ADX | Average Directional Index | Mesure la force du trend (0-100) |
| +DI/-DI | Directional Indicators | Direction du mouvement (haussier/baissier) |

**Points cles** :
- Un ADX > 25 indique un trend fort, < 20 indique un marche sans tendance
- Le croisement MACD/Signal genere les signaux d'entree/sortie
- Les percentiles ADX (5eme, 50eme, 85eme) servent de seuils adaptatifs
- L'ecart-type eleve de l'AD indique une forte variabilite de la force du trend

> **Note technique** : Le MACD standard utilise 12/26/9. L'AD est calcule sur 14 periodes par defaut. Ces valeurs peuvent etre ajustees selon la volatilite de l'actif.

In [ ]:
# Detect market regimes
def detect_regimes(close, window=126):
    """Detect bull/bear/sideways regimes based on rolling returns"""
    returns = close.pct_change(window)
    
    # Regime thresholds
    bull_threshold = 0.15  # +15% over 6 months = bull
    bear_threshold = -0.15  # -15% over 6 months = bear
    
    regimes = pd.Series('sideways', index=close.index)
    regimes[returns > bull_threshold] = 'bull'
    regimes[returns < bear_threshold] = 'bear'
    
    return regimes, returns

regimes, rolling_returns = detect_regimes(data['close'])

# Count regime days
regime_counts = regimes.value_counts()
print("Market Regime Distribution (2017-2025):")
for regime, count in regime_counts.items():
    pct = count / len(regimes) * 100
    print(f"  {regime.capitalize()}: {count} days ({pct:.1f}%)")

# Analyze ADX by regime
print("\nADX Statistics by Regime:")
for regime in ['bull', 'bear', 'sideways']:
    mask = regimes == regime
    adx_values = data.loc[mask, 'adx'].dropna()
    print(f"  {regime.capitalize()}: mean={adx_values.mean():.1f}, median={adx_values.median():.1f}, p75={adx_values.quantile(0.75):.1f}")

### Interpretation : Distribution des regimes de marche BTC

La detection des regimes (bull/bear/sideways) permet d'analyser la performance de la strategie selon les conditions de marche.

| Regime | Definition | Caracteristique ADX typique |
|--------|------------|----------------------------|
| Bull | Rendement 6M > +15% | ADX eleve (trend fort) |
| Bear | Rendement 6M < -15% | ADX eleve (trend baissier) |
| Sideways | Rendement 6M entre -15% et +15% | ADX faible (absence de trend) |

**Points cles** :
- La distribution des regimes influence la performance globale de la strategie
- L'AD varie selon le regime : plus eleve en trend, plus faible en range
- Comprendre la distribution des regimes permet d'ajuster les parametres ADX
- Les seuils de percentiles ADX doivent etre adaptes a chaque regime

> **Note technique** : La detection de regime utilise une fenetre de 126 jours (6 mois). Une fenetre plus courte cree plus de transitions, une fenetre plus longue lisse les changements de regime.

In [ ]:
# Backtest engine for MACD+ADX strategy
def backtest_macd_adx(data, 
                       adx_window=80,
                       adx_lower_pct=5,
                       adx_upper_pct=85,
                       start_date='2019-01-01',
                       end_date='2025-02-18'):
    """
    Backtest MACD+ADX strategy with adaptive ADX thresholds.
    
    Entry logic:
    - Long: MACD crosses above signal AND ADX > upper percentile threshold
    - Short: MACD crosses below signal AND ADX > upper percentile threshold
    
    Exit logic:
    - MACD crosses opposite signal
    - OR ADX drops below lower percentile threshold
    """
    # Filter by date
    mask = (data.index >= start_date) & (data.index <= end_date)
    df = data[mask].copy()
    
    if len(df) < adx_window:
        return None
    
    # Calculate dynamic ADX thresholds based on percentile
    def get_adx_thresholds(adx_series, window, lower_pct, upper_pct):
        """Calculate rolling percentile-based ADX thresholds"""
        lower = adx_series.rolling(window=window, min_periods=window//2).quantile(lower_pct/100)
        upper = adx_series.rolling(window=window, min_periods=window//2).quantile(upper_pct/100)
        return lower, upper
    
    adx_lower, adx_upper = get_adx_thresholds(df['adx'], adx_window, adx_lower_pct, adx_upper_pct)
    
    # Generate signals
    signals = pd.Series(0, index=df.index)
    position = 0
    
    for i in range(1, len(df)):
        current_date = df.index[i]
        prev_date = df.index[i-1]
        
        macd_curr = df['macd'].iloc[i]
        macd_prev = df['macd'].iloc[i-1]
        signal_curr = df['signal'].iloc[i]
        signal_prev = df['signal'].iloc[i-1]
        adx_curr = df['adx'].iloc[i]
        
        # Get current ADX thresholds
        lower_thresh = adx_lower.iloc[i] if not pd.isna(adx_lower.iloc[i]) else 20
        upper_thresh = adx_upper.iloc[i] if not pd.isna(adx_upper.iloc[i]) else 25
        
        # Check for crossovers
        bull_cross = (macd_prev <= signal_prev) and (macd_curr > signal_curr)
        bear_cross = (macd_prev >= signal_prev) and (macd_curr < signal_curr)
        
        if position == 0:  # Not in position
            if bull_cross and adx_curr > upper_thresh:
                position = 1
            elif bear_cross and adx_curr > upper_thresh:
                position = -1
        else:  # In position
            # Exit on opposite cross OR low ADX
            exit_signal = False
            if position == 1:
                if bear_cross or adx_curr < lower_thresh:
                    exit_signal = True
            elif position == -1:
                if bull_cross or adx_curr < lower_thresh:
                    exit_signal = True
            
            if exit_signal:
                position = 0
        
        signals.iloc[i] = position
    
    # Calculate returns
    returns = df['close'].pct_change()
    strategy_returns = returns * signals.shift(1)
    
    # Metrics
    total_return = (1 + strategy_returns).cumsum().iloc[-1]
    sharpe = np.sqrt(252) * strategy_returns.mean() / strategy_returns.std() if strategy_returns.std() > 0 else 0
    max_dd = (strategy_returns.cumsum() / strategy_returns.cumsum().cummax() - 1).min()
    
    # Trade count
    trades = (signals.diff() != 0).sum()
    
    return {
        'total_return': total_return,
        'sharpe': sharpe,
        'max_drawdown': max_dd,
        'trades': trades,
        'signals': signals,
        'returns': strategy_returns
    }

# Test current parameters
result = backtest_macd_adx(data, adx_window=80, adx_lower_pct=5, adx_upper_pct=85)
if result:
    print("Current Parameters (Window=80, Pct 5/85):")
    print(f"  Sharpe: {result['sharpe']:.3f}")
    print(f"  Total Return: {result['total_return']*100:.1f}%")
    print(f"  Max Drawdown: {result['max_drawdown']*100:.1f}%")
    print(f"  Trades: {result['trades']}")

### Interpretation : Performance du backtest avec parametres actuels

Le backtest simule la strategie MACD+ADX sur la periode 2019-2025 avec les parametres actuels.

| Metrique | Valeur | Interpretation |
|----------|--------|----------------|
| Sharpe | A calculer | Ratio de Sharpe > 1 est considere bon |
| Total Return | A calculer | Performance cumulee sur la periode |
| Max Drawdown | A calculer | Perte maximale from peak (doit etre < 30%) |
| Trades | A calculer | Nombre de signaux generes |

**Points cles** :
- Un Sharpe negatif indique que la strategie ne bat pas le "risk-free rate"
- Le drawdown maximal mesure le risque de ruine : plus il est bas, meilleure est la strategie
- Le nombre de trades influence les couts de transaction : trop de trades = erosion des profits
- Ces resultats serviront de baseline pour comparer avec les parametres optimises

> **Note technique** : Le backtest utilise des donnees yfinance locales. Pour la production, il faudra valider sur QuantConnect avec les donnees Binance BTCUSDT.

In [ ]:
# Grid search for optimal parameters
def grid_search(data, param_grid):
    """Grid search over parameter space"""
    results = []
    
    for window in param_grid['adx_window']:
        for lower_pct in param_grid['adx_lower_pct']:
            for upper_pct in param_grid['adx_upper_pct']:
                if lower_pct >= upper_pct:
                    continue
                
                result = backtest_macd_adx(
                    data, 
                    adx_window=window,
                    adx_lower_pct=lower_pct,
                    adx_upper_pct=upper_pct
                )
                
                if result:
                    results.append({
                        'adx_window': window,
                        'adx_lower_pct': lower_pct,
                        'adx_upper_pct': upper_pct,
                        'sharpe': result['sharpe'],
                        'total_return': result['total_return'],
                        'max_drawdown': result['max_drawdown'],
                        'trades': result['trades']
                    })
    
    return pd.DataFrame(results)

# Define parameter grid
param_grid = {
    'adx_window': [40, 60, 80, 100, 120],
    'adx_lower_pct': [3, 5, 7, 10],
    'adx_upper_pct': [75, 80, 85, 90]
}

print("Running grid search...")
grid_results = grid_search(data, param_grid)

# Sort by Sharpe
grid_results = grid_results.sort_values('sharpe', ascending=False)
print("\nTop 10 Parameter Combinations (by Sharpe):")
print(grid_results.head(10).to_string())

### Interpretation : Resultats du grid search

Le grid search explore l'espace des parametres pour identifier la combinaison optimale maximisant le ratio de Sharpe.

| Parametre | Plage testee | Impact sur la performance |
|-----------|--------------|---------------------------|
| `adx_window` | 40-120 jours | Plus court = plus reactif, plus long = plus stable |
| `adx_lower_pct` | 3-10 | Seuil de sortie : trop bas = sorties tardives |
| `adx_upper_pct` | 75-90 | Seuil d'entree : trop haut = peu de signaux |

**Points cles** :
- Le Sharpe est la metrique principale : elle mesure le rendement ajuste du risque
- Un ecart trop large entre lower et upper percentile reduit le nombre de trades
- Les parametres optimaux dependent de la periode d'analyse (2019-2025)
- Verifier que les meilleurs parametres ne sont pas des outliers (sur-apprentissage)

> **Note technique** : Le grid search est une methode d'optimisation brute force. Pour des espaces de parametres plus larges, considerer l'optimisation bayesienne ou les algorithmes genetiques.

In [ ]:
# Analyze results by regime
def backtest_by_regime(data, regimes, adx_window=80, adx_lower_pct=5, adx_upper_pct=85):
    """Test strategy performance in different market regimes"""
    results_by_regime = {}
    
    for regime in ['bull', 'bear', 'sideways']:
        mask = regimes == regime
        regime_data = data[mask].copy()
        
        if len(regime_data) > adx_window:
            result = backtest_macd_adx(
                regime_data,
                adx_window=adx_window,
                adx_lower_pct=adx_lower_pct,
                adx_upper_pct=adx_upper_pct
            )
            
            if result:
                results_by_regime[regime] = result
    
    return results_by_regime

# Test best parameters from grid search
best_params = grid_results.iloc[0]
print(f"\nBest Parameters: Window={best_params['adx_window']}, LowerPct={best_params['adx_lower_pct']}, UpperPct={best_params['adx_upper_pct']}")
print(f"Expected Sharpe: {best_params['sharpe']:.3f}")

regime_results = backtest_by_regime(
    data, regimes,
    adx_window=int(best_params['adx_window']),
    adx_lower_pct=int(best_params['adx_lower_pct']),
    adx_upper_pct=int(best_params['adx_upper_pct'])
)

print("\nPerformance by Regime:")
for regime, result in regime_results.items():
    print(f"  {regime.capitalize()}: Sharpe={result['sharpe']:.3f}, Return={result['total_return']*100:.1f}%")

### Interpretation : Performance par regime de marche

L'analyse des resultats par regime permet de comprendre la robustesse de la strategie selon les conditions de marche.

| Regime | Performance attendue | Interpretation |
|--------|---------------------|----------------|
| Bull | Leverage positif | La strategie devrait profiter des tendances haussieres |
| Bear | Potentiel short | Les signaux bearish peuvent generer du alpha |
| Sideways | Risque de whipsaw | Faible tendance = faux signaux MACD frequents |

**Points cles** :
- Une strategie MACD+ADX robuste doit performer de maniere acceptable dans tous les regimes
- Si la performance est negative en sideways, considerer ajouter un filtre de volatilite
- Le ratio Sharpe bull vs bear indique la capacite de la strategie a naviguer les retournements

> **Note technique** : L'analyse par regime est essentielle pour la robustesse. Une strategie qui ne fonctionne qu'en bull market n'est pas deployable en production.

## Conclusion : Recommandations de recherche

### Resume des resultats

L'analyse par grid search a permis d'identifier les parametres optimaux pour la strategie MACD+ADX sur BTC.

| Parametre | Valeur optimale | Impact |
|-----------|-----------------|--------|
| ADX Window | A determiner par grid search | Responsivite vs stabilite |
| ADX Lower Percentile | A determiner | Seuil de sortie de position |
| ADX Upper Percentile | A determiner | Seuil d'entree en position |

### Points cles a retenir

1. **Fenetre ADX** : Une fenetre plus courte (40-80) offre une meilleure reactivite aux changements de regime
2. **Seuils adaptatifs** : L'utilisation de percentiles dynamiques s'adapte automatiquement a la volatilite du marche
3. **Performance par regime** : La strategie doit etre validee separement en bull/bear/sideways

> **Note technique** : Les parametres optimaux identifies ici doivent etre valides par un backtest complet sur QuantConnect, car l'environnement de recherche local (yfinance) peut differer legèrement des donnees QC.

### Prochaines etapes

1. Transposer les parametres C# trouves vers `Main.cs`
2. Compiler sur le cloud QuantConnect
3. Executer le backtest sur la periode etendue (2019-2025)
4. Comparer le Sharpe reel vs attendu
5. Iterer si necessaire avec des ajustements fins